In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tomkin import detect_rpeak
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from outlier_calculation import Quality,compute_outlier_ecg
# from hrvanalysis import remove_ectopic_beats
from joblib import Parallel,delayed
from data_quality import ECGQualityCalculation
from joblib import delayed,Parallel
from copy import deepcopy
from ecg import ecg_feature_computation
from scipy import interpolate
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler,RobustScaler,QuantileTransformer
import gzip

In [23]:
def rip_cycle_feature_computation(peaks_datastream: np.ndarray,
                                  valleys_datastream: np.ndarray) -> np.ndarray:
    """
    Respiration Feature Implementation. The respiration feature values are
    derived from the following paper:
    'puffMarker: a multi-sensor approach for pinpointing the timing of first lapse in smoking cessation'
    Removed due to lack of current use in the implementation
    roc_max = []  # 8. ROC_MAX = max(sample[j]-sample[j-1])
    roc_min = []  # 9. ROC_MIN = min(sample[j]-sample[j-1])

    :param peaks_datastream: list of peak datapoints
    :param valleys_datastream: list of valley datapoints
    :return: lists of DataPoints each representing a specific feature calculated from the respiration cycle
    found from the peak valley inputs
    """

    inspiration_duration = []  # 1 Inhalation duration
    expiration_duration = []  # 2 Exhalation duration
    respiration_duration = []  # 3 Respiration duration
    inspiration_expiration_ratio = []  # 4 Inhalation and Exhalation ratio
    stretch = []  # 5 Stretch
    upper_stretch = []  # 6. Upper portion of the stretch calculation
    lower_stretch = []  # 7. Lower portion of the stretch calculation
    delta_previous_inspiration_duration = []  # 10. BD_INSP = INSP(i)-INSP(i-1)
    delta_previous_expiration_duration = []  # 11. BD_EXPR = EXPR(i)-EXPR(i-1)
    delta_previous_respiration_duration = []  # 12. BD_RESP = RESP(i)-RESP(i-1)
    delta_previous_stretch_duration = []  # 14. BD_Stretch= Stretch(i)-Stretch(i-1)
    delta_next_inspiration_duration = []  # 19. FD_INSP = INSP(i)-INSP(i+1)
    delta_next_expiration_duration = []  # 20. FD_EXPR = EXPR(i)-EXPR(i+1)
    delta_next_respiration_duration = []  # 21. FD_RESP = RESP(i)-RESP(i+1)
    delta_next_stretch_duration = []  # 23. FD_Stretch= Stretch(i)-Stretch(i+1)
    neighbor_ratio_expiration_duration = []  # 29. D5_EXPR(i) = EXPR(i) / avg(EXPR(i-2)...EXPR(i+2))
    neighbor_ratio_stretch_duration = []  # 32. D5_Stretch = Stretch(i) / avg(Stretch(i-2)...Stretch(i+2))

    valleys = valleys_datastream
    peaks = peaks_datastream[:-1]

    for i, peak in enumerate(peaks):
        valley_start_time = valleys[i][0]
        valley_end_time = valleys[i + 1][0]

        delta = peak[0] - valleys[i][0]
        inspiration_duration.append(np.array([valley_start_time,valley_end_time,delta/1000]))

        delta = valleys[i + 1][0] - peak[0]
        expiration_duration.append(np.array([valley_start_time,valley_end_time,delta/1000]))

        delta = valleys[i + 1][0] - valley_start_time
        respiration_duration.append(np.array([valley_start_time,valley_end_time,delta/1000]))

        ratio = (peak[0] - valley_start_time) / (valleys[i + 1][0] - peak[0])
        inspiration_expiration_ratio.append(np.array([valley_start_time,valley_end_time,ratio]))

        value = peak[1] - valleys[i + 1][1]
        stretch.append(np.array([valley_start_time,valley_end_time,value]))

    for i, point in enumerate(inspiration_duration):
        valley_start_time = valleys[i][0]
        valley_end_time = valleys[i + 1][0]
        if i == 0:  # Edge case
            delta_previous_inspiration_duration.append(np.array([valley_start_time,valley_end_time,0]))
            delta_previous_expiration_duration.append(np.array([valley_start_time,valley_end_time,0]))
            delta_previous_respiration_duration.append(np.array([valley_start_time,valley_end_time,0]))
            delta_previous_stretch_duration.append(np.array([valley_start_time,valley_end_time,0]))
        else:
            delta = inspiration_duration[i][2] - inspiration_duration[i - 1][2]
            delta_previous_inspiration_duration.append(np.array([valley_start_time,valley_end_time,delta]))

            delta = expiration_duration[i][2] - expiration_duration[i - 1][2]
            delta_previous_expiration_duration.append(np.array([valley_start_time,valley_end_time,delta]))

            delta = respiration_duration[i][2] - respiration_duration[i - 1][2]
            delta_previous_respiration_duration.append(np.array([valley_start_time,valley_end_time,delta]))

            delta = stretch[i][2] - stretch[i - 1][2]
            delta_previous_stretch_duration.append(np.array([valley_start_time,valley_end_time,delta]))

        if i == len(inspiration_duration) - 1:
            delta_next_inspiration_duration.append(np.array([valley_start_time,valley_end_time,0]))
            delta_next_expiration_duration.append(np.array([valley_start_time,valley_end_time,0]))
            delta_next_respiration_duration.append(np.array([valley_start_time,valley_end_time,0]))
            delta_next_stretch_duration.append(np.array([valley_start_time,valley_end_time,0]))
        else:
            delta = inspiration_duration[i][2] - inspiration_duration[i + 1][2]
            delta_next_inspiration_duration.append(np.array([valley_start_time,valley_end_time,delta]))

            delta = expiration_duration[i][2] - expiration_duration[i + 1][2]
            delta_next_expiration_duration.append(np.array([valley_start_time,valley_end_time,delta]))

            delta = respiration_duration[i][2] - respiration_duration[i + 1][2]
            delta_next_respiration_duration.append(np.array([valley_start_time,valley_end_time,delta]))

            delta = stretch[i][2] - stretch[i + 1][2]
            delta_next_stretch_duration.append(np.array([valley_start_time,valley_end_time,delta]))

        stretch_average = 0
        expiration_average = 0
        count = 0.0
        for j in [-2, -1, 1, 2]:
            if i + j < 0 or i + j >= len(inspiration_duration):
                continue
            stretch_average += stretch[i + j][2]
            expiration_average += expiration_duration[i + j][2]
            count += 1

        stretch_average /= count
        expiration_average /= count

        ratio = stretch[i][2] / stretch_average
        neighbor_ratio_stretch_duration.append(np.array([valley_start_time,valley_end_time,ratio]))

        ratio = expiration_duration[i][2] / expiration_average
        neighbor_ratio_expiration_duration.append(np.array([valley_start_time,valley_end_time,ratio]))

    # Begin assembling datastream for output
    inspiration_duration_datastream = np.array(inspiration_duration)[1:-1]

    expiration_duration_datastream = np.array(expiration_duration)[1:-1]

    respiration_duration_datastream = np.array(respiration_duration)[1:-1]

    inspiration_expiration_ratio_datastream = np.array(inspiration_expiration_ratio)[1:-1]

    stretch_datastream = np.array(stretch)[1:-1]

    delta_previous_inspiration_duration_datastream = np.array(delta_previous_inspiration_duration)[1:-1]

    delta_previous_expiration_duration_datastream = np.array(delta_previous_expiration_duration)[1:-1]

    delta_previous_respiration_duration_datastream = np.array(delta_previous_respiration_duration)[1:-1]

    delta_previous_stretch_duration_datastream = np.array(delta_previous_stretch_duration)[1:-1]

    delta_next_inspiration_duration_datastream = np.array(delta_next_inspiration_duration)[1:-1]

    delta_next_expiration_duration_datastream = np.array(delta_next_expiration_duration)[1:-1]

    delta_next_respiration_duration_datastream = np.array(delta_next_respiration_duration)[1:-1]

    delta_next_stretch_duration_datastream = np.array(delta_next_stretch_duration)[1:-1]

    neighbor_ratio_expiration_datastream = np.array(neighbor_ratio_expiration_duration)[1:-1]

    neighbor_ratio_stretch_datastream = np.array(neighbor_ratio_stretch_duration)[1:-1]

    return np.concatenate([inspiration_duration_datastream,
                           expiration_duration_datastream[:,2].reshape(-1,1),
                           respiration_duration_datastream[:,2].reshape(-1,1),
                           inspiration_expiration_ratio_datastream[:,2].reshape(-1,1),
                           stretch_datastream[:,2].reshape(-1,1),
                           delta_previous_inspiration_duration_datastream[:,2].reshape(-1,1),
                           delta_previous_expiration_duration_datastream[:,2].reshape(-1,1),
                           delta_previous_respiration_duration_datastream[:,2].reshape(-1,1),
                           delta_previous_stretch_duration_datastream[:,2].reshape(-1,1),
                           delta_next_inspiration_duration_datastream[:,2].reshape(-1,1),
                           delta_next_expiration_duration_datastream[:,2].reshape(-1,1),
                           delta_next_respiration_duration_datastream[:,2].reshape(-1,1),
                           delta_next_stretch_duration_datastream[:,2].reshape(-1,1),
                           neighbor_ratio_expiration_datastream[:,2].reshape(-1,1),
                           neighbor_ratio_stretch_datastream[:,2].reshape(-1,1)],axis=1)

def get_windows(data,window_size=10,offset=10,fs=1):
    ts_array = np.arange(data[0,0],data[-1,0],offset*1000)
    window_col = []
    for t in ts_array:
        index = np.where((data[:,0]>t-window_size*1000/2)&(data[:,0]<=t+window_size*1000/2))[0]
        if len(index)<30:
            continue
        window_col.append(data[index,:])
    return window_col

def get_std_chest(window,start=1,end=4):
    return np.array([np.mean(window[:,0]),np.sqrt(np.sum(np.power(np.std(window[:,start:end],axis=0),2)))])


def filter_ecg_windows(ecg_windows,acl_std):
    final_ecg_windows = []
    for window in ecg_windows:
        index = np.where((acl_std[:,0]>window[0,0])&(acl_std[:,0]<window[-1,0]))[0]
        if len(index)==0:
            continue
        window_temp = acl_std[index,1].reshape(-1)
        if len(window_temp[window_temp>.21])/len(window_temp) > .5:
            continue
        final_ecg_windows.append(window)
    return final_ecg_windows

def get_rip_windows(data,window_size=60,offset=10,fs=.2):
    ts_array = np.arange(data[0,0],data[-1,0],offset*1000)
    window_col = []
    for t in ts_array:
        index = np.where((data[:,0]>=t-window_size*1000/2)&(data[:,1]<=t+window_size*1000/2))[0]
        if len(index)<10:
            continue
        window_col.append(data[index,:])
    return window_col

def get_all_windows(data,rip_data,window_size=60,offset=10,fs=.2):
    ts_array = np.arange(data[0,0],data[-1,0],offset*1000)
    window_col = []
    for t in ts_array:
        index = np.where((data[:,0]>=t-window_size*1000/2)&(data[:,0]<=t+window_size*1000/2))[0]
        index_rip = np.where((rip_data[:,0]>=t-window_size*1000/2)&(rip_data[:,1]<=t+window_size*1000/2))[0]
        if len(index)<30 or len(index_rip)<10:
            continue
        window_col.append([data[index,:],rip_data[index_rip,:]])
    return window_col
def get_features(a):
    window = a[0]
    ecg_feature = list([window[0,0],window[-1,0]]+ecg_feature_computation(window[:,0],window[:,1]))
    window = a[1]
    rip_feature = list(np.mean(window[:,2:],axis=0))+list(np.std(window[:,2:],axis=0))+ \
    list(np.percentile(window[:,2:],80,axis=0))+list(np.percentile(window[:,2:],20,axis=0))
    feature = list([window[0,0],window[-1,0]])+rip_feature
    return np.array(feature)
    

path = './data/'
participants = [path + f +'/' for f in os.listdir(path) if f[0]=='S']
for f in participants:
    if 'ecg.txt.gz' not in os.listdir(f):
        continue
    if os.path.isfile(f+'ecg_rr.p') and os.path.isfile(f+'pv.p') :
        ecg_rr = pickle.load(open(f+'ecg_rr.p','rb'))
        ecg_rr_baseline = ecg_rr
        from scipy import stats
        ecg_rr[:,1] = stats.mstats.winsorize(ecg_rr[:,1],limits=.1)
        
        peaks,valleys = pickle.load(open(f+'pv.p','rb'))
        rip_features = rip_cycle_feature_computation(peaks,valleys)
        rip_features = rip_features[:,np.array([0,1,2,3,4,5,6,-2,-1])]
        for c in range(2,rip_features.shape[1]):
            rip_features[:,c] = stats.mstats.winsorize(rip_features[:,c],limits=.1)
            
        all_windows = get_all_windows(ecg_rr,rip_features,window_size=60,offset=30,fs=1)
        all_features = np.array(list(map(lambda a:get_features(a),all_windows)))
        for i in range(2,all_features.shape[1],1):
            all_features[:,i] = StandardScaler().fit_transform(all_features[:,i].reshape(-1,1)).reshape(-1)
        pickle.dump(all_features,open(f+'features_rip1.p','wb'))
        print(all_features.shape)

(236, 30)
(252, 30)
(240, 30)
(253, 30)
(272, 30)
(112, 30)
(264, 30)
(258, 30)
(251, 30)
(202, 30)
(270, 30)
(255, 30)
(219, 30)
(254, 30)
(225, 30)
(252, 30)
(210, 30)


In [24]:
# Soujanya Chatterjee
	
# 2:06 PM (9 minutes ago)
	
# to me
import pandas as pd, numpy as np, os, csv, glob, math, matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from datetime import datetime
from scipy.stats import *
import gzip
import pickle
from collections import Counter

def find_majority(k):
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    for n in k:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1

        # Keep track of maximum on the go
        if myMap[n] > maximum[1]: maximum = (n,myMap[n])

    return maximum[0]

# _dir = 'W:\\Students\\cstress_features\\data\\data\\SI02\\'

def decodeLabel(label):
    label = label[:2]  # Only the first 2 characters designate the label code

    mapping = {'c1': 0, 'c2': 1, 'c3': 1, 'c4': 0, 'c5': 0, 'c6': 0, 'c7': 2}

    return mapping[label]

def readstressmarks(participantID, filename):
    features = []
    for file in os.listdir(filename):    
        if file.endswith("marks.txt.gz"):        
            with gzip.open(os.path.join(filename, file), 'r') as file:
                for line in file.readlines():
                    line = line.decode('utf8').strip()
                    parts = [x.strip() for x in line.split(',')]                    
                    label = parts[0][:2]  
                    if label not in ['c7','c6']:
                        stressClass = decodeLabel(label)
                        features.append([participantID, stressClass, int(parts[2]), int(parts[3])])
    return np.array(features)
_dirr = './data/'
parti = np.array(os.listdir(_dirr) )
header = ['participant','starttime','endtime','label'] + ['f_'+str(i) for i in range(28)]
fea_cols = ['f_'+str(i) for i in range(28)]
data = []
for p in parti:
    print(p)
    if p in ['feature.csv','feature_ecg.csv','feature_rip.csv','feature_all.csv',
             '.ipynb_checkpoints']:
        continue
    else:
        if os.path.isdir(os.path.join(_dirr,p)):
            _dir = (os.path.join(_dirr,p))
            gt_marks = readstressmarks(p,_dir)
            if len(gt_marks)==0:
                continue
            groundtruth = pd.DataFrame({'participant': gt_marks[:, 0], 'label': gt_marks[:, 1], 'starttime': gt_marks[:, 2],
                                        'endtime': gt_marks[:, 3]}, columns=['participant','label','starttime','endtime'])
            groundtruth = groundtruth.sort_values('starttime')
   
            check = False
            for file in os.listdir(_dir):    
                    if file.endswith('features_rip1.p'):                    
                        with open(_dir+'/'+file, 'rb') as f:  
                            x = pickle.load(f)
                            check  =True
            if not check:
                continue
#             print(x.shape)
#             dataset = pd.DataFrame({'starttime': x[:, 0], 'endtime': x[:, 1], 'f_1': x[:, 2]
#                                    , 'f_2': x[:, 3], 'f_3': x[:, 4], 'f_4': x[:, 5]
#                                    , 'f_5': x[:, 6], 'f_6': x[:, 7], 'f_7': x[:, 8]
#                                    , 'f_8': x[:, 9], 'f_9': x[:, 10], 'f_10': x[:, 11]
#                                    , 'f_11': x[:, 12]}, columns=['starttime','endtime','f_1','f_2','f_3','f_4','f_5','f_6','f_7','f_8',
#                                                                  'f_9','f_10','f_11'])
            
            dataset = pd.DataFrame(x,columns=['starttime','endtime']+['f_'+str(i) for i in range(x.shape[1]-2)])
            dataset = dataset.sort_values('starttime')

            for gt in range(len(dataset)):
                starttime = int(dataset['starttime'].iloc[gt])
                endtime = int(dataset['endtime'].iloc[gt])
                result = []
                for line in range(len(groundtruth)):
                    id, gtt, st, et = [groundtruth['participant'].iloc[line], groundtruth['label'].iloc[line], int(groundtruth['starttime'].iloc[line]),
                                      int(groundtruth['endtime'].iloc[line])]
                    if starttime < st:
                        continue
                    else:
                        if (starttime > st) and (endtime < et):
                            result.append(gtt)
                        if result:
                            fea = list(dataset[fea_cols].iloc[gt])
                            inter_data = [p, st,et,find_majority(result)],(fea)
                            flatten = lambda l: [item for sublist in l for item in sublist]

                            data.append(flatten(inter_data))
    #         print(data)
df = pd.DataFrame(data)
print(df.shape)
df.to_csv(_dirr + '/' + 'feature_rip.csv', index=False, header=header)
df.shape

.DS_Store
SI01
SI02
SI03
SI04
SI05
SI06
SI07
SI08
SI09
SI10
SI11
SI12
SI13
SI14
SI15
SI16
SI17
SI18
SI19
SI20
SI21
SI22
SI23
SI24
.ipynb_checkpoints
feature.csv
feature_rip.csv
feature_ecg.csv
feature_all.csv
(2290, 32)


(2290, 32)

In [25]:
feature_file = './data/feature_rip.csv'
feature = pd.read_csv(feature_file).values
y = np.int64(feature[:,3])
X = feature[:,4:]
groups = feature[:,0]
print(X.shape,y.shape,np.sum(y),np.unique(y),len(np.unique(groups)))

(2290, 28) (2290,) 447 [0 1] 17


In [26]:
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
m = len(np.where(y==0)[0])
n = len(np.where(y>0)[0])
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import Parallel,delayed

delta = 0.1

paramGrid = {
             'pca__n_components':[4,5,6],
             'rf__kernel': ['rbf'],
             'rf__C': np.logspace(.1,4,3),
             'rf__gamma': [np.power(2,np.float(x)) for x in np.arange(-4, 0, .5)],
             'rf__class_weight': [{0: w, 1: 1 - w} for w in [.4,.3,.2]],
             'rf__probability':[True]
}

paramGrid = {
             'kernel': ['rbf'],
             'C': np.logspace(.1,4,3),
             'gamma': [np.power(2,np.float(x)) for x in np.arange(-4, 0, .5)],
             'class_weight': [{0: w, 1: 1 - w} for w in [.4,.3,.2]],
             'probability':[True]
}
# clf = Pipeline([('pca',PCA()),('rf', SVC())])
clf = SVC()
gkf = GroupKFold(n_splits=len(np.unique(groups)))
grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y,groups=groups)),
                           scoring='f1_weighted',verbose=5)
grid_search.fit(X,y)

print("Best parameter (CV score=%0.3f):" % grid_search.best_score_)
print(grid_search.best_params_)

Fitting 17 folds for each of 72 candidates, totalling 1224 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 552 tasks      | elapsed:   38.9s
[Parallel(n_jobs=-1)]: Done 786 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 1056 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1224 out of 1224 | elapsed:  1.2min finished


Best parameter (CV score=0.895):
{'C': 1.2589254117941673, 'class_weight': {0: 0.3, 1: 0.7}, 'gamma': 0.08838834764831845, 'kernel': 'rbf', 'probability': True}


In [27]:
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
clf = grid_search.best_estimator_
y_pred = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups))
print(confusion_matrix(y,y_pred),classification_report(y,y_pred))

[[1682  161]
 [  81  366]]               precision    recall  f1-score   support

           0       0.95      0.91      0.93      1843
           1       0.69      0.82      0.75       447

    accuracy                           0.89      2290
   macro avg       0.82      0.87      0.84      2290
weighted avg       0.90      0.89      0.90      2290



In [28]:
import pickle
print(clf)
clf.fit(X,y)
pickle.dump(clf,open('/home/jupyter/mullah/cc3/rip_model_feature_standardization_minnesota.p','wb'))

SVC(C=1.2589254117941673, break_ties=False, cache_size=200,
    class_weight={0: 0.3, 1: 0.7}, coef0=0.0, decision_function_shape='ovr',
    degree=3, gamma=0.08838834764831845, kernel='rbf', max_iter=-1,
    probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [ ]:
from sklearn_porter import Porter
porter = Porter(clf, language='java')
output = porter.export(export_data=True)
# print(output)
text_file = open("SVM.java", "w")
text_file.write(output)
text_file.close()
print(clf.probA_,clf.probB_)